# Credit Default Swaps

## Credit curves

Just like a discount curve is a way of representing the underlying interest rates implicit in the market quotes of a collection of real-world interest rate products, **credit curves** are a way of representing the data implied by credit default swaps.

**Credit default swaps** (**CDS**) are instruments whose value depends on the likelihood that a given company (the curve's **issuer**) will suffer a credit event over a given period.

A **credit event** can be a default, the failure to make payments, the issuer entering into bankruptcy proceedings, or the occurence of other legal events. The exact definition of what constitutes a credit event depends on a series of factors and is usually defined in some kind of ISDA (International Swaps and Derivatives Association) master agreement.

In any case, we will generically call a credit event a *default*, and talk about **non-default probabilities** (**NDP**), i.e. the probability that the issuer will not suffer a credit event before a given date. 

NDPs are the equivalent for credit curves of discount factors for discount curves. Just like discount curves, credit curves are built by specifying a pricing/observation date, a sequence of pillar dates and a sequence of NDPs. We will then implement a $\tt{CreditCurve}$ class that provides a method which interpolates between the pillars of the curve to return the NDP at an arbitrary value date between the pricing date and the last pillar date.

In addition, we'll also write a method which returns the **hazard rate** at an arbitrary value date. The hazard rate is the credit curve equivalent of the short rate or overnight rate for discount curves. It represents the instantaneous probability of the issuer defaulting conditioned on it not having defaulted until that moment. In practice we will calculate it numerically, and therefore it'll be the (annualized) conditional probability of the issuer defaulting between the value date and the day after.

<table>
  <tr width=200>
    <th>Discount Curve</th>
    <th>Credit Curve</th>
  </tr>
  <tr>
    <td>Represents underlying rates implicit in market quotes of IR products</td>
    <td>Represents default probability implied by credit default swaps</td>
  </tr>
  <tr>
    <td>discount factors</td>
    <td>non-default probabilities</td>
  </tr>
  <tr>
    <td>short rate</td>
    <td>hazard rate</td>
  </tr>
</table>   

### Hazard Rate

Hazard rate is often called a *conditional failure rate* since it's expression is a direct
application of the conditional probability concept.

Conditional probability answers to the question "how should you update probabilities of events when there is additional information available ?". To derive the general formula let's start with an example.

A fair die is rolled. Let $A$ be the event that the outcome is an odd number ($A={1,3,5}$). Also let $B$ be the event that the outcome is less than or equal to $3$ ($B={1,2,3}$). What is the probability of $A$ ($P(A)$) ? What is the probability of $A$ given $B$ ($P(A|B)$) ?

Being a simple example we can compute the result by hand:

$$P(A) = \frac{|A|}{|S|} = \frac{|\{1,3,5\}|}{6} = \frac{1}{2}\qquad\textrm{(where S is the entire sample space)}$$

Now let's find the conditional probability of $A$ given that $B$ occurred. If we know $B$ has occurred, the outcome must be among $\{1,2,3\}$. For $A$ to also happen the outcome must be in $A\cap B = \{1,3\}$. Since all die rolls are equally likely, we argue that $P(A|B)$ must be equal to

$$P(A|B) = \frac{|A\cap B|}{|B|} = \frac{2}{3}$$

To generalize our example we can rewrite the calculation by dividing the numerator and denominator by the entire space of the events $|S|$ hence:

$$P(A|B) = \cfrac{|A\cap B|}{|B|} = \cfrac{\cfrac{|A\cap B|}{|S|}}{\cfrac{|B|}{|S|}} = \cfrac{P(A\cap B)}{P(B)}$$

<img src="conditional_b.png" width=500>

In formula if the non-default probability is indicated by $N$ and the hazard rate by $\lambda$:

$$\lambda = -\cfrac{dN}{dt}\cfrac{1}{N(t_0, t_1)}$$

where the minus sign derives from the fact that $N$ is a **non** default probability while the hazard rate is defined in terms of the probability of default.

Conversly given the hazard rate the non-default probability can be determined as:

$$\lambda = -\cfrac{1}{dt}\cdot\cfrac{dN}{N} = -\cfrac{d(\textrm{log}N)}{dt}$$

$$N(t_0, t) = e^{-\int_{t_0}^{t}\lambda dt}$$

In [ ]:
import math, numpy
from dateutil.relativedelta import relativedelta

class CreditCurve(object):
    
    def __init__(self, pillar_dates, pillar_ndps):
        self.pillar_dates = pillar_dates
        
        self.pillar_days = [
            (pd - pillar_dates[0]).days
            for pd in pillar_dates
        ]
        
        self.log_ndps = [
            math.log(ndp)
            for ndp in pillar_ndps
        ]
        
    def ndp(self, value_date):
        value_days = (value_date - self.pillar_dates[0]).days
        return math.exp(
            numpy.interp(value_days,
                         self.pillar_days,
                         self.log_ndps))
    
    def hazard(self, value_date):
        ndp_1 = self.ndp(value_date)
        ndp_2 = self.ndp(value_date + relativedelta(days=1))
        delta_t = 1.0 / 365.0
        h = -1.0 / ndp_1 * (ndp_2 - ndp_1) / delta_t
        return h

As usual we test the newly developed class with some dummy data.

In [ ]:
from datetime import date

pricing_date = date.today()

cc = CreditCurve(
    [pricing_date, pricing_date + relativedelta(years=2)],
    [1.0, 0.8]
)

In [ ]:
cc.ndp(pricing_date + relativedelta(years=1))

In [ ]:
cc.hazard(pricing_date + relativedelta(years=1))

## Credit Deafult Swaps

Once we have implemented a $\tt{CreditCurve}$ class which allows us to interpolate pillar non-default probabilities (NDPs), and also to calculate the hazard rate at arbitrary dates, we can use it to price **credit default swaps** (CDSs).

CDSs are made up of two legs:

* the *default* leg: which pays $LGD = 1 - R$, known as the **loss given default**, if and when the credit event occurs. A conventional value for the recovery parameter $R$ is 40%;
* the *premium* leg: which pays a *spread* $S$ every m months until the credit event occurs.

### Premium leg

Let's start with the premium leg, which is easier to calculate. We will use the following notation:

* $t$ today's date;
* $t_0$ the start date of the CDS (could be different from $t$);
* $t_1, ..., t_n$ the payment dates of the premium leg, which occur at a m-month frequency (we assume that $t_n$ is the end date of the CDS);
* $D(t')$ the discount factor between $t$ and $t'$;
* $N(t')$ the non-default probability between $t$ and $t'$;
* $\tau$ the random variable representing the date of the credit event.

At each payment date $t_i$, a flow $S$ is paid if and only if the credit event has not occurred before that date. Therefore the NPV of the each flow is

$$\mathbb{E}\left[ S \times D(t_i) \times \mathbb{1}(\tau > t_i) \right] = S \cdot D(t_i) \cdot N(t_i)$$
therefore the NPV of the premium leg is simply the sum, over the payment dates, of these terms:

$$\textrm{NPV}_{premium} = \sum_{i=1}^{n} S \cdot D(t_i) \cdot N(t_i)$$

## Default leg

The LGD $(1-R)$ is paid out on the same date on which the credit event occurs, i.e. it can potentially be paid out on any date between $t_0$ and $t_n$. Mathematically, therefore, the NPV of the premium leg can be expressed as follows:

$$ \mathbb{E} \left[(1-R) \times D(\tau) \times \mathbb{1}(\tau \leq t_n) \right] $$

Using the law of total probability, we can break this down into the sum of "daily NPVs" calculated as a function of the daily "forward" default probabilities $\mathbb{P}$:

$$
\begin{align*}
\mathbb{E}\left[(1-R) \times D(\tau) \times \mathbb{1}(\tau \leq t_n) \right]
&= \sum_{t'=t_0}^{t_n} \mathbb{E}[ (1-R) \times D(\tau) | \tau = t'] \mathbb{P}[ \tau = t' ] \\
&= (1-R) \sum_{t'=t_0}^{t_n} D(t') \left( \mathbb{P}[ \tau \geq t' ] - \mathbb{P}[ \tau \geq t'+1 ] \right) \\
&= (1-R) \sum_{t'=t_0}^{t_n} D(t') \left( N(t') - N(t'+1) \right)
\end{align*}
$$

where the last step holds since $\mathbb{P}[\tau\geq d^{'}] = 1 - \mathbb{P}[\tau < d^{'}] = 1 - (1-N(d^{'})) = N(d^{'})$.

In [ ]:
from finmarkets import generate_swap_dates

class CreditDefaultSwap:
    
    def __init__(self, notional, start_date, fixed_spread, 
                 maturity, tenor=3, recovery=0.4):
        self.notional = notional
        self.payment_dates = generate_swap_dates(start_date, maturity*12, tenor)
        self.fixed_spread = fixed_spread
        self.recovery = recovery
    
    def premium_leg_npv(self, discount_curve, credit_curve):
        npv = 0
        for i in range(1, len(self.payment_dates)):
            npv += (
                self.fixed_spread *
                discount_curve.df(self.payment_dates[i]) *
                credit_curve.ndp(self.payment_dates[i])
            )
        return npv * self.notional
    
    def default_leg_npv(self, discount_curve, credit_curve):
        npv = 0
        d = self.payment_dates[0]
        while d <= self.payment_dates[-1]:
            npv += discount_curve.df(d) * (
                credit_curve.ndp(d) -
                credit_curve.ndp(d + relativedelta(days=1))
            )
            d += relativedelta(days=1)
        return npv * self.notional * (1 - self.recovery)
    
    def npv(self, discount_curve, credit_curve):
        return self.default_leg_npv(discount_curve, credit_curve) - \
               self.premium_leg_npv(discount_curve, credit_curve)

Below a simple test of the class.

In [ ]:

credit_curve = CreditCurve([pricing_date, pricing_date + relativedelta(months=36)], 
                           [1.0, 0.7])

cds = CreditDefaultSwap(1e6, pricing_date, 3, 0.03)
cds.premium_leg_npv(discount_curve, credit_curve)

In [ ]:
cds.default_leg_npv(discount_curve, credit_curve)

In [ ]:
cds.npv(discount_curve, credit_curve)

## Estimate Default Probabilities from CDS

Pretty much like the discount curves could be derived from swap market quotes, we can estimate default probabilities (hence credit curves) from CDS quotes.
Following the same steps outlined in Chapter~\ref{swaps-and-bootstrapping---practical-lesson-5} in fact, through a boostrapping algorithm, we can determine default probabilites at discrete dates to fill our curve.

Although the derivation is left as an exercise to the reader here we list the necessary steps:

* collect market quotes for a number of CDS with different maturities;
* create the corresponding CDS objects;
* define a $\tt{CreditCurve}$ whose pillars are the CDS maturity dates and with a set of unknown default probabilities;
* define an objective function to minimize the sum of the squared CDS's NPVs;
* set the non-default probabilities to an initial value and define their range of variability between $[0, 1]$ since they are probabilities and fix "today's" probability to 1 since there hasn't been any default;
* run the minimization.
